# Apple - Supplier Performance Metrics

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_supplier = pd.read_csv('Data/006/suppliers.csv')
df_deliveries = pd.read_csv('Data/006/supplier_deliveries.csv')

df_supplier.head()

,supplier_id,supplier_name
0,101,Alpha Components
1,102,Beta Electronics
2,103,Gamma Materials
3,104,Delta Manufacturing
4,105,Epsilon Systems


In [3]:
df_deliveries.head()

,supplier_id,delivery_date,component_count,manufacturing_region
0,101,2024-10-05,1500,North America
1,102,2024-10-12,2200,Asia
2,103,2024-10-19,1800,Europe
3,101,2024-10-26,1200,North America
4,104,2024-10-08,2500,Asia


# Pregunta 1

### Necesitamos saber quiénes son nuestros proveedores más activos. Identifica a los 5 principales proveedores basándote en el volumen total de componentes entregados en octubre de 2024.

In [9]:
df_merged = pd.merge(df_supplier, df_deliveries, on='supplier_id')

df_octubre = df_merged[
    (df_merged['delivery_date'].between('2024-10-01','2024-10-31'))
]

respuesta1 = df_octubre.groupby('supplier_name')['component_count'].sum().reset_index()

respuesta1 = respuesta1.sort_values('component_count', ascending=False).head(5)

respuesta1

,supplier_name,component_count
1,Beta Electronics,4300
5,Gamma Materials,3700
0,Alpha Components,2700
2,Delta Manufacturing,2500
4,Eta Technologies,2300


```SQL
SELECT
    s.supplier_name,
    SUM(d.component_count) AS total_delivered
FROM supplier_deliveries d
JOIN suppliers s ON d.supplier_id = s.supplier_id
WHERE d.delivery_date BETWEEN '2024-10-01' AND '2024-10-31'
GROUP BY s.supplier_name
ORDER BY total_delivered DESC
LIMIT 5;
```

# Pregunta 2

### Para cada región, encuentra el ID del proveedor que entregó el mayor número de componentes en noviembre de 2024. Esto nos ayudará a entender qué proveedor está manejando el mayor volumen por mercado.

In [16]:
df_deliveries['delivery_date'] = pd.to_datetime(df_deliveries['delivery_date'])
resumen = df_deliveries[df_deliveries['delivery_date'].dt.month == 11].groupby(['manufacturing_region','supplier_id'])['component_count'].sum().reset_index()

In [14]:
resumen = resumen.sort_values('component_count', ascending=False)

respuesta = resumen.drop_duplicates(subset='manufacturing_region')

respuesta

,manufacturing_region,supplier_id,component_count
6,North America,108,7300
2,Asia,110,5100
4,Europe,109,2800


```SQL
 SELECT 
    manufacturing_region, 
    supplier_id, 
    total_components
FROM (
    SELECT 
        manufacturing_region, 
        supplier_id, 
        SUM(component_count) AS total_components,
        RANK() OVER (PARTITION BY manufacturing_region ORDER BY SUM(component_count) DESC) as ranking
    FROM supplier_deliveries
    WHERE delivery_date BETWEEN '2024-11-01' AND '2024-11-30'
    GROUP BY manufacturing_region, supplier_id
) AS subquery
WHERE ranking = 1;
```

# Pregunta 3

### Necesitamos identificar posibles brechas en nuestra cadena de suministro para Asia. Haz una lista de los nombres de todos los proveedores que no han entregado ningún componente a la región de fabricación 'Asia' en diciembre de 2024.

In [18]:
si_entregaron = df_deliveries[
    (df_deliveries['manufacturing_region'] == 'Asia') &
    (df_deliveries['delivery_date'].between('2024-12-01','2024-12-31'))
]['supplier_id'].unique()

no_entregaron = df_supplier[~df_supplier['supplier_id'].isin(si_entregaron)]

respuesta = no_entregaron['supplier_name']

respuesta

0        Alpha Components
2         Gamma Materials
3     Delta Manufacturing
4         Epsilon Systems
5          Zeta Solutions
7       Theta Innovations
8         Iota Processing
9          Kappa Supplies
10            Lambda Corp
11              Mu Global
Name: supplier_name, dtype: object

```SQL
SELECT
    s.supplier_name
FROM suppliers s
LEFT JOIN supplier_deliveries d ON s.supplier_id = d.supplier_id
AND d.manufacturing_region = 'Asia'
AND d.delivery_date BETWEEN '2024-12-01' AND '2024-12-31'
WHERE d.supplier_id IS NULL
```

# 